In [60]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Concatenate, TimeDistributed, RepeatVector
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 데이터 불러오기
df = pd.read_csv('../../../Data/using/dt_2.6/ByAstar_dt_2.6_1016.csv')

df = df.dropna(subset=['combination'])

# 데이터셋 분리: train_set 8 : test_set 2
train_df, test_df = train_test_split(df, shuffle=True, test_size=0.2)
# train_set을 다시 8:2로 나눠서 train_set과 validation_set을 만듦
train_df, val_df = train_test_split(train_df, shuffle=True, test_size=0.2)
# random_state=42

# 모든 시퀀스의 길이 중에서 최대 길이를 구하기
all_sequences = [eval(str(seq)) for seq in df['combination'].values]
max_seq_length = max([len(seq) for seq in all_sequences])

# 각 데이터셋에서 theta, phi, sequence 추출하고 reshape 적용
theta_train = train_df['Theta'].values.reshape(-1, 1)
phi_train = train_df['Phi'].values.reshape(-1, 1)
sequence_train = pad_sequences(train_df['combination'].apply(eval).tolist(), maxlen=max_seq_length, padding='pre')

theta_val = val_df['Theta'].values.reshape(-1, 1)
phi_val = val_df['Phi'].values.reshape(-1, 1)
sequence_val = pad_sequences(val_df['combination'].apply(eval).tolist(), maxlen=max_seq_length, padding='pre')

theta_test = test_df['Theta'].values.reshape(-1, 1)
phi_test = test_df['Phi'].values.reshape(-1, 1)
sequence_test = pad_sequences(test_df['combination'].apply(eval).tolist(), maxlen=max_seq_length, padding='pre')


In [61]:
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


# 하이퍼파라미터 탐색 공간 정의
space = {
    'hidden_units': hp.quniform('hidden_units', 32, 256, 32),  # 은닉 유닛 수 (32에서 256까지 32의 배수)
    'learning_rate': hp.loguniform('learning_rate', -6, -3),  # 로그 스케일 학습률
    'optimizer': hp.choice('optimizer', ['adam', 'sgd', 'rmsprop']),
#     'epochs': hp.quniform('epochs', 50, 1000, 10),  # 에포크 (50에서 200까지 10의 배수)
    'batch_size': hp.quniform('batch_size', 16, 256, 16),  # 배치 크기 (16에서 256까지 16의 배수)
    'dropout': hp.uniform('dropout', 0.0, 0.5),  # 드롭아웃 확률 (0에서 0.5)
    'l1_regularization': hp.uniform('l1_regularization', 0.0, 0.1),  # L1 정규화
    'l2_regularization': hp.uniform('l2_regularization', 0.0, 0.1)  # L2 정규화
}

# Objective 함수 정의
def objective(params):
     
     # 모델 정의
     theta_input = Input(shape=(1,), name='theta_input')
     phi_input = Input(shape=(1,), name='phi_input')

     # theta와 phi를 Concatenate
     merged = Concatenate()([theta_input, phi_input])

     # 시퀀스를 예측하기 위한 GRU 레이어
     repeated_vector = RepeatVector(max_seq_length)(merged)  # max_sequence_length는 시퀀스의 최대 길이

     rnn_layer = GRU(int(params['hidden_units']), return_sequences=True, name='rnn_layer')(repeated_vector)
     rnn_layer = Dropout(params['dropout'])(rnn_layer)  # 드롭아웃 레이어 추가

     output = TimeDistributed(Dense(5, activation='softmax'), name='output_layer')(rnn_layer)

     model = Model(inputs=[theta_input, phi_input], outputs=output)

     optimizer = params['optimizer']
     
     if optimizer == 'adam':
          optimizer = Adam(learning_rate=params['learning_rate'])
     elif optimizer == 'sgd':
          optimizer = SGD(learning_rate=params['learning_rate'])
     elif optimizer == 'rmsprop':
          optimizer = RMSprop(learning_rate=params['learning_rate'])
          
     # 컴파일 및 훈련
     model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

     # Early Stopping 콜백 설정
     early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
    
     history = model.fit([theta_train, phi_train], np.expand_dims(sequence_train, -1),
                         validation_data=([theta_val, phi_val], np.expand_dims(sequence_val, -1)),
                         epochs=10000, batch_size=int(params['batch_size']), verbose=1)

     # 검증
     loss, accuracy = model.evaluate([theta_test, phi_test], np.expand_dims(sequence_test, -1), verbose=0)

     return {'loss': loss, 'status': STATUS_OK}


# Hyperopt를 이용한 랜덤 탐색
trials = Trials()

best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)
print(best)


  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

job exception: No module named 'keras.src.engine.base_layer_v1'



  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]


ModuleNotFoundError: No module named 'keras.src.engine.base_layer_v1'